In [1]:
from ExtractFeatures import extract_features
from GaborFilter.GaborFilter import generate_gabor_kernel_cv
import skimage as ski

In [3]:
kernels = generate_gabor_kernel_cv()
img = ski.io.imread("../data/processed/fonts-dataset/Scheherazade New/4.jpeg")
features = extract_features(img,kernels)
print(features)
print(len(features))

[8.72544298e-02 1.21762367e+02 1.26306373e+02 1.88529600e+06
 1.14388104e+02 1.23731535e+02 4.66887900e+06 9.74927786e+01
 1.15107654e+02 1.20768060e+07 7.24713432e+01 9.76139767e+01
 2.11529450e+07 1.22066079e+02 1.26262737e+02 1.91107900e+06
 1.15829422e+02 1.23926878e+02 4.64338400e+06 1.06691550e+02
 1.19747747e+02 8.65795000e+06 9.03646033e+01 1.10225734e+02
 1.55666240e+07 1.21975407e+02 1.26656691e+02 1.36438400e+06
 1.18498479e+02 1.25571116e+02 2.67717800e+06 1.06213459e+02
 1.20382883e+02 7.80093800e+06 8.97053342e+01 1.11556404e+02
 1.39592850e+07 1.22583756e+02 1.26333279e+02 1.84906800e+06
 1.15612559e+02 1.24021889e+02 4.47106600e+06 1.03143219e+02
 1.17937974e+02 1.01401270e+07 8.32155915e+01 1.05169120e+02
 1.83810210e+07]
49


<mark> below are some test code the GLCM  and LBP <mark>

In [3]:
# from skimage.feature import graycomatrix, graycoprops
# from skimage import io, color ,img_as_ubyte
# from scipy.stats import entropy
# # Load the image
# image = io.imread('../../data/raw/fonts-dataset/Scheherazade New/0.jpeg')
# image2 = io.imread('../../data/raw/fonts-dataset/Marhey/0.jpeg')

# # Convert the image to grayscale
# gray_image = color.rgb2gray(image)
# gray_image2 = color.rgb2gray(image2)
# gray_image = img_as_ubyte(gray_image)
# gray_image2 = img_as_ubyte(gray_image2)
# # Compute the GLCM
# glcm = graycomatrix(gray_image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
# glcm2 = graycomatrix(gray_image2, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

# # Compute GLCM properties
# contrast = graycoprops(glcm, 'contrast')[0, 0]
# dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
# homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
# energy = graycoprops(glcm, 'energy')[0, 0]
# correlation = graycoprops(glcm, 'correlation')[0, 0]


# zcontrast = graycoprops(glcm2, 'contrast')[0, 0]
# zdissimilarity = graycoprops(glcm2, 'dissimilarity')[0, 0]
# zhomogeneity = graycoprops(glcm2, 'homogeneity')[0, 0]
# zenergy = graycoprops(glcm2, 'energy')[0, 0]
# zcorrelation = graycoprops(glcm2, 'correlation')[0, 0]

# print('Contrast:', contrast)
# print('Dissimilarity:', dissimilarity)
# print('Homogeneity:', homogeneity)
# print('Energy:', energy)
# print('Correlation:', correlation)
# entropy_val = entropy(glcm.flatten())

# print('Entropy:', entropy_val)


# print('Contrast:', zcontrast)
# print('Dissimilarity:', zdissimilarity)
# print('Homogeneity:', zhomogeneity)
# print('Energy:', zenergy)
# print('Correlation:', zcorrelation)
# entropy_val = entropy(glcm2.flatten())

# print('Entropy:', entropy_val)





In [4]:
# from skimage import io
# from skimage.feature import local_binary_pattern
# from skimage.color import rgb2gray
# import matplotlib.pyplot as plt

# # Load the image
# image = io.imread('../../data/processed/fonts-dataset/Lemonada/920.jpeg')

# # Convert the image to grayscale

# # Settings for LBP
# radius = 3
# n_points = 8 * radius

# # Compute LBP
# lbp = local_binary_pattern(image, n_points, radius, method="uniform")

# # Display the original image and LBP
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ax1.imshow(image, cmap='gray')
# ax1.set_title('Original Image')
# ax1.axis('off')

# ax2.imshow(lbp, cmap='gray')
# ax2.set_title('LBP Image')
# ax2.axis('off')

# plt.show()